Seek his will in all you do, and he will show you which path to take.
Proverbs 3:6

Use QAOA to verify the partitioning of neurons by cortical depth.


## Data Loading

In [1]:
import numpy as np
import pandas as pd
from numba import njit
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
from scipy import stats, signal
from scipy.spatial.distance import pdist, squareform
from scipy.cluster import hierarchy
from scipy.stats import binomtest, binned_statistic 
from scipy.optimize import curve_fit
from sklearn.decomposition import PCA, NMF
from sklearn.cluster import DBSCAN, KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mutual_info_score  
from sklearn.metrics import silhouette_score
import networkx as nx
from tqdm import tqdm
import pickle

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Import custom modules
from pathlib import Path
import sys
PROJECT_ROOT = Path().resolve().parent  
sys.path.insert(0, str(PROJECT_ROOT))

from src.dataio.config import DATA_DIR, FILESYSTEM_CONFIG
from src.dataio.data_structures import SessionData
from src.dataio.loaders import load_session_complete
from src.dataio.processors import ResponseProcessor, NetworkAnalyzer, DimensionalityReducer
from src.dataio.validators import validate_session_comprehensive, analyze_session_quality
from src import features

📁 Data directory: /Volumes/pack/qbraid/neuroscience-qaoa/notebooks/data
✓ Data directory exists: True


In [2]:
DATA_DIR = Path('../data/raw')

# Find all subdirs that contain the expected .mat file
MAT_FILENAME = 'allPlanesVariables27-Feb-2021.mat'
sessions = sorted([
    d.name
    for d in DATA_DIR.iterdir()
    if d.is_dir() and (d / MAT_FILENAME).is_file()
])

print(f"Discovered {len(sessions)} sessions:")
for s in sessions:
    print("  ", s)

# Choose your primary session here (must be one of the discovered names)
PRIMARY_SESSION = '081820_355n'
if PRIMARY_SESSION not in sessions:
    raise ValueError(f"Primary session '{PRIMARY_SESSION}' not found in data/. Available: {sessions}")

print(f"\nPrimary analysis session: {PRIMARY_SESSION}")

# --- now load & process as before ---

session_path = DATA_DIR / PRIMARY_SESSION
print(f"Loading session from: {session_path}")

session = load_session_complete(session_path, use_cache=True)

validation_result = validate_session_comprehensive(session)
quality_report     = analyze_session_quality(session)

print(f"\nSession loaded successfully!")
print(f"Neurons: {len(session.neurons)}")
print(f"Trials:  {len(session.trials)}")
print(f"Activity matrix shape: {session.activity_matrix.shape}")
print(f"Data quality score: {quality_report['overall_score']:.2f}")


Discovered 9 sessions:
   031020_367n_100um20st_FRA
   031020_367r_100um20st_FRA
   031120_352ll_100um20st_FRA
   031120_352ll_100um20st_FRA_diffxy
   080720_400r
   081820_355n
   081920_355l
   081920_355r
   082620_355l

Primary analysis session: 081820_355n
Loading session from: ../data/raw/081820_355n


No signal correlation matrix was found in any source.
No noise correlation matrix was found in any source.
Trimmed trials to 22 to match available data
VALIDATION WARNING: Unexpected trial count: 22. Expected one of [90, 180]
VALIDATION WARNING: Activity matrix contains 4 NaN values (0.00%).



Session loaded successfully!
Neurons: 416
Trials:  22
Activity matrix shape: (416, 2060)
Data quality score: 1.00
